In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, sum, avg

In [3]:
spark = SparkSession.builder \
        .appName("Fake Friends") \
        .getOrCreate()

24/07/30 14:19:34 WARN Utils: Your hostname, Aayushs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.29.173 instead (on interface en0)
24/07/30 14:19:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/30 14:19:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.csv("fakefriends.csv", inferSchema=True)
df.show()

+---+--------+---+---+
|_c0|     _c1|_c2|_c3|
+---+--------+---+---+
|  0|    Will| 33|385|
|  1|Jean-Luc| 26|  2|
|  2|    Hugh| 55|221|
|  3|  Deanna| 40|465|
|  4|   Quark| 68| 21|
|  5|  Weyoun| 59|318|
|  6|  Gowron| 37|220|
|  7|    Will| 54|307|
|  8|  Jadzia| 38|380|
|  9|    Hugh| 27|181|
| 10|     Odo| 53|191|
| 11|     Ben| 57|372|
| 12|   Keiko| 54|253|
| 13|Jean-Luc| 56|444|
| 14|    Hugh| 43| 49|
| 15|     Rom| 36| 49|
| 16|  Weyoun| 22|323|
| 17|     Odo| 35| 13|
| 18|Jean-Luc| 45|455|
| 19|  Geordi| 60|246|
+---+--------+---+---+
only showing top 20 rows



In [5]:
df.createGlobalTempView("fakefriends")

In [6]:
df2 = df.withColumnRenamed("_c0","Index") \
            .withColumnRenamed("_c1","Name") \
            .withColumnRenamed("_c2","Age") \
            .withColumnRenamed("_c3","Number")
df2.show(1)

+-----+----+---+------+
|Index|Name|Age|Number|
+-----+----+---+------+
|    0|Will| 33|   385|
+-----+----+---+------+
only showing top 1 row



In [7]:
# Perform the aggregation
df3 = df2.groupBy("Age").agg(sum("Number").alias("age_sum")).show()

+---+-------+
|Age|age_sum|
+---+-------+
| 31|   2138|
| 65|   1491|
| 53|   1560|
| 34|   1473|
| 28|   2091|
| 26|   4115|
| 27|   1825|
| 44|   3386|
| 22|   1445|
| 47|   2099|
| 52|   3747|
| 40|   4264|
| 20|    825|
| 57|   3106|
| 54|   3615|
| 48|   2814|
| 19|   2346|
| 64|   3376|
| 41|   2417|
| 43|   1614|
+---+-------+
only showing top 20 rows



In [8]:
# Perform the aggregation
df3 = df2.groupBy("Age").agg(avg("Number").alias("age_avg")).show()

+---+------------------+
|Age|           age_avg|
+---+------------------+
| 31|            267.25|
| 65|             298.2|
| 53|222.85714285714286|
| 34|             245.5|
| 28|             209.1|
| 26|242.05882352941177|
| 27|           228.125|
| 44| 282.1666666666667|
| 22|206.42857142857142|
| 47|233.22222222222223|
| 52| 340.6363636363636|
| 40| 250.8235294117647|
| 20|             165.0|
| 57| 258.8333333333333|
| 54| 278.0769230769231|
| 48|             281.4|
| 19|213.27272727272728|
| 64| 281.3333333333333|
| 41|268.55555555555554|
| 43|230.57142857142858|
+---+------------------+
only showing top 20 rows



In [13]:
age_count = df2.groupBy("Age").count().orderBy("Age").show()

+---+-----+
|Age|count|
+---+-----+
| 18|    8|
| 19|   11|
| 20|    5|
| 21|    8|
| 22|    7|
| 23|   10|
| 24|    5|
| 25|   11|
| 26|   17|
| 27|    8|
| 28|   10|
| 29|   12|
| 30|   11|
| 31|    8|
| 32|   11|
| 33|   12|
| 34|    6|
| 35|    8|
| 36|   10|
| 37|    9|
+---+-----+
only showing top 20 rows



# SPARK SQL

In [9]:
# Drop table if exists
spark.sql("DROP TABLE IF EXISTS global_temp.fakefriends")

# Create Global Temporary View
df2.createGlobalTempView("fakefriends")

In [10]:
# Check if the view is created successfully
spark.sql("SELECT * FROM global_temp.fakefriends").show()

+-----+--------+---+------+
|Index|    Name|Age|Number|
+-----+--------+---+------+
|    0|    Will| 33|   385|
|    1|Jean-Luc| 26|     2|
|    2|    Hugh| 55|   221|
|    3|  Deanna| 40|   465|
|    4|   Quark| 68|    21|
|    5|  Weyoun| 59|   318|
|    6|  Gowron| 37|   220|
|    7|    Will| 54|   307|
|    8|  Jadzia| 38|   380|
|    9|    Hugh| 27|   181|
|   10|     Odo| 53|   191|
|   11|     Ben| 57|   372|
|   12|   Keiko| 54|   253|
|   13|Jean-Luc| 56|   444|
|   14|    Hugh| 43|    49|
|   15|     Rom| 36|    49|
|   16|  Weyoun| 22|   323|
|   17|     Odo| 35|    13|
|   18|Jean-Luc| 45|   455|
|   19|  Geordi| 60|   246|
+-----+--------+---+------+
only showing top 20 rows



In [11]:
spark.sql("select * from global_temp.fakefriends where Age >= 13 and Age <= 19").show()

+-----+-------+---+------+
|Index|   Name|Age|Number|
+-----+-------+---+------+
|   21|  Miles| 19|   268|
|   52|Beverly| 19|   269|
|   54|  Brunt| 19|     5|
|  106|Beverly| 18|   499|
|  115|  Dukat| 18|   397|
|  133|  Quark| 19|   265|
|  136|   Will| 19|   335|
|  225|   Elim| 19|   106|
|  304|   Will| 19|   404|
|  341|   Data| 18|   326|
|  366|  Keiko| 19|   119|
|  373|  Quark| 19|   272|
|  377|Beverly| 18|   418|
|  404| Kasidy| 18|    24|
|  409|    Nog| 19|   267|
|  439|   Data| 18|   417|
|  444|  Keiko| 18|   472|
|  492|  Dukat| 19|    36|
|  494| Kasidy| 18|   194|
+-----+-------+---+------+



# approach 2

In [18]:
age_count = df2.select("*").where("Age >= 13 and Age <= 19").orderBy("Age").show()

+-----+-------+---+------+
|Index|   Name|Age|Number|
+-----+-------+---+------+
|  106|Beverly| 18|   499|
|  115|  Dukat| 18|   397|
|  341|   Data| 18|   326|
|  377|Beverly| 18|   418|
|  404| Kasidy| 18|    24|
|  439|   Data| 18|   417|
|  444|  Keiko| 18|   472|
|  494| Kasidy| 18|   194|
|   21|  Miles| 19|   268|
|   52|Beverly| 19|   269|
|   54|  Brunt| 19|     5|
|  133|  Quark| 19|   265|
|  136|   Will| 19|   335|
|  225|   Elim| 19|   106|
|  304|   Will| 19|   404|
|  366|  Keiko| 19|   119|
|  373|  Quark| 19|   272|
|  409|    Nog| 19|   267|
|  492|  Dukat| 19|    36|
+-----+-------+---+------+

